In [ ]:
#from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
import random
import tensorflow as tf

from gensim.models import Word2Vec

In [ ]:
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential 
#from tensorflow.keras.models import Sequential, load_model
#from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras import backend

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import LancasterStemmer

nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
data = pd.read_csv("/content/drive/My Drive/NLP/Project/NLPMiniProject_data.csv")
print(data.head())

           ID                                            comment     date  \
0  uid_590555  Well, let's be honest here, they don't actuall...  2015-04   
1  uid_671762  Well, I didn't need evidence to believe in com...  2016-12   
2  uid_519689              Who does an "official promo" in 360p?  2013-11   
3  uid_788362                           Grotto koth was the best  2015-09   
4  uid_299252                                   Neal's back baby  2015-11   

   down                                     parent_comment  score  top  \
0     0  They should shut the fuck up and let the commu...      2    2   
1    -1  You need evidence to kill people? I thought we...      6   -1   
2     0                    2014 BMW S1000R: Official Promo      3    3   
3     0  Not really that memorable lol if you want memo...      2    2   
4     0                      James Neal hit on Zach Parise     -5   -5   

              topic            user  label  
0       starcitizen  Combat_Wombatz      0  
1 

In [ ]:
df = data[['ID','comment','parent_comment','down','score','top','topic','label']]

In [ ]:
df['comment'][0]

"Well, let's be honest here, they don't actually seem to do much moderating, so they have to spend their time doing *something*."

In [ ]:
df['parent_comment'][0]

'They should shut the fuck up and let the community decide what it wants for the subreddit. They are here to **moder** fucking **ate**, not to dictate.'

In [ ]:
(df['comment']+df['parent_comment'])[0]

"Well, let's be honest here, they don't actually seem to do much moderating, so they have to spend their time doing *something*.They should shut the fuck up and let the community decide what it wants for the subreddit. They are here to **moder** fucking **ate**, not to dictate."

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['comment']+df['parent_comment'],
    df['label'],
    test_size=0.2, 
    random_state=42
)

In [ ]:
X_train.shape, X_test.shape

((12000,), (3000,))

In [ ]:
desired_vocab_size = 10000 #Vocablury size
t = tf.keras.preprocessing.text.Tokenizer(num_words=desired_vocab_size, oov_token=32) # num_words -> Vocablury size

In [ ]:
#Fit tokenizer with actual training data
t.fit_on_texts(X_train.tolist())

In [ ]:
#Vocabulary
t.word_index

{32: 1,
 'the': 2,
 'to': 3,
 'a': 4,
 'and': 5,
 'i': 6,
 'of': 7,
 'is': 8,
 'you': 9,
 'that': 10,
 'in': 11,
 'it': 12,
 'for': 13,
 'this': 14,
 'on': 15,
 'be': 16,
 'but': 17,
 'with': 18,
 'have': 19,
 'not': 20,
 'are': 21,
 'they': 22,
 'was': 23,
 'just': 24,
 'so': 25,
 'my': 26,
 'if': 27,
 'like': 28,
 'he': 29,
 'what': 30,
 'as': 31,
 'all': 32,
 "it's": 33,
 'or': 34,
 'at': 35,
 'would': 36,
 'do': 37,
 'people': 38,
 'your': 39,
 'get': 40,
 'me': 41,
 'we': 42,
 'can': 43,
 "don't": 44,
 'about': 45,
 'no': 46,
 'from': 47,
 'one': 48,
 'because': 49,
 'an': 50,
 'out': 51,
 'up': 52,
 "i'm": 53,
 'how': 54,
 'will': 55,
 'there': 56,
 'more': 57,
 'when': 58,
 'who': 59,
 'them': 60,
 'has': 61,
 'their': 62,
 'think': 63,
 'know': 64,
 'his': 65,
 'by': 66,
 'only': 67,
 'some': 68,
 'time': 69,
 'good': 70,
 'really': 71,
 'why': 72,
 'game': 73,
 'than': 74,
 'well': 75,
 'even': 76,
 'now': 77,
 'then': 78,
 'should': 79,
 'had': 80,
 'him': 81,
 'been': 82,
 '

# Prepare Training and Test Data

Get the word index for each of the word in the review

In [ ]:
X_train[0:1]

9839    "black shield", xpecial confirmed racist and i...
dtype: object

In [ ]:
X_train = t.texts_to_sequences(X_train.tolist())

In [ ]:
print(X_train[0:1])

[[295, 1630, 9908, 1232, 459, 5, 4922, 45, 65, 1, 171, 3646, 9908, 414, 729]]


In [ ]:
X_test = t.texts_to_sequences(X_test)

# Pad Sequences - Important

In [ ]:
#Define maximum number of words to consider in each review
max_review_length = 300

In [ ]:
#Pad training and test reviews
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train,
                                                        maxlen=max_review_length,
                                                        padding='pre', 
                                                        truncating='post')

X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, 
                                                       maxlen=max_review_length, 
                                                       padding='pre',
                                                       truncating='post')

In [ ]:
X_train.shape,X_test.shape

((12000, 300), (3000, 300))

# Build the Graph

In [ ]:
#Initialize model
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

Add Embedding layer
 - Embedding Layer Input = Batch_Size * Length of each review

In [ ]:
model.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    50, #Embedding size
                                    input_length=max_review_length) #Number of words in each review
          )

In [ ]:
model.output

<KerasTensor: shape=(None, 300, 50) dtype=float32 (created by layer 'embedding')>

Embedding Layer Output - [Batch_Size , Review Length , Embedding_Size]

Add LSTM Layer with 256 as RNN state size

In [ ]:
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(128)) #RNN State - size of cell state and hidden state

In [ ]:
model.output

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'lstm')>

In [ ]:
model.add(tf.keras.layers.Dropout(0.25))

Use Dense layer for output layer

In [ ]:
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [ ]:
#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 50)           500050    
                                                                 
 dropout (Dropout)           (None, 300, 50)           0         
                                                                 
 lstm (LSTM)                 (None, 128)               91648     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 591,827
Trainable params: 591,827
Non-trainable params: 0
_________________________________________________________________


# Execute the graph

In [ ]:
model.fit(X_train,y_train,
          epochs=5,
          batch_size=128,          
          validation_data=(X_test, y_test))

Epoch 1/5
94/94 [==============================] - 108s 1s/step - loss: 0.6880 - accuracy: 0.5405 - val_loss: 0.6822 - val_accuracy: 0.5637
Epoch 2/5
94/94 [==============================] - 109s 1s/step - loss: 0.6391 - accuracy: 0.6366 - val_loss: 0.6651 - val_accuracy: 0.5990
Epoch 3/5
94/94 [==============================] - 102s 1s/step - loss: 0.5134 - accuracy: 0.7521 - val_loss: 0.7475 - val_accuracy: 0.5903
Epoch 4/5
94/94 [==============================] - 101s 1s/step - loss: 0.3884 - accuracy: 0.8244 - val_loss: 0.8742 - val_accuracy: 0.5787
Epoch 5/5
94/94 [==============================] - 101s 1s/step - loss: 0.2953 - accuracy: 0.8733 - val_loss: 0.9843 - val_accuracy: 0.5750


Pre-Trained Embeddings

In [ ]:
import gensim.downloader as api
import numpy as np

In [ ]:
glove_model = api.load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded


In [1]:
#Size of the model
glove_model.vectors.shape

NameError: ignored

In [ ]:
#Embedding for word great
glove_model['and']

array([ 0.26818 ,  0.14346 , -0.27877 ,  0.016257,  0.11384 ,  0.69923 ,
       -0.51332 , -0.47368 , -0.33075 , -0.13834 ,  0.2702  ,  0.30938 ,
       -0.45012 , -0.4127  , -0.09932 ,  0.038085,  0.029749,  0.10076 ,
       -0.25058 , -0.51818 ,  0.34558 ,  0.44922 ,  0.48791 , -0.080866,
       -0.10121 , -1.3777  , -0.10866 , -0.23201 ,  0.012839, -0.46508 ,
        3.8463  ,  0.31362 ,  0.13643 , -0.52244 ,  0.3302  ,  0.33707 ,
       -0.35601 ,  0.32431 ,  0.12041 ,  0.3512  , -0.069043,  0.36885 ,
        0.25168 , -0.24517 ,  0.25381 ,  0.1367  , -0.31178 , -0.6321  ,
       -0.25028 , -0.38097 ], dtype=float32)

In [ ]:
#Initialize embedding matrix for our dataset with 10000+1 rows (1 for padding word)
#and 50 columns (as embedding size is 50)
embedding_matrix = np.zeros((desired_vocab_size + 1, 50))

In [ ]:
for word, i in sorted(t.word_index.items(),key=lambda x:x[1]):
    if i > (desired_vocab_size+1):
        break
    try:
        embedding_vector = glove_model[word] #Reading word's embedding from Glove model for a given word
        embedding_matrix[i] = embedding_vector
    except:
        pass

In [ ]:
embedding_matrix[200]

array([ 0.29262999,  0.31020001, -0.12961   ,  0.35857999,  0.59204   ,
        0.46941   , -0.024792  , -0.30658999,  0.20464   ,  0.77373999,
       -0.14436001,  0.42105001,  0.030358  ,  0.01449   ,  0.39918   ,
        0.38801   ,  0.098906  ,  0.39401001,  0.10773   , -0.95802999,
        0.041812  ,  0.044068  , -0.24857999,  0.0076151 ,  0.38519999,
       -1.63950002, -0.68910998,  0.43243   ,  0.93482   , -0.33768001,
        2.9611001 ,  0.15918   , -0.19574   ,  0.12766001,  0.21367   ,
       -0.2272    , -0.21646   ,  0.41707   , -0.023072  , -0.74263   ,
        0.28744999,  0.46162   ,  0.18258999,  0.019844  ,  0.18715   ,
        0.54868001,  0.24677999,  0.29536   , -0.0064379 ,  0.80203003])

Build a Model with Pretained Embedding

In [ ]:
#Initialize model
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

In [ ]:
model.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    50, #Embedding size
                                    weights=[embedding_matrix],
                                    trainable=False,
                                    input_length=max_review_length) #Number of words in each review
          )

In [ ]:
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(128)) #RNN State - size of cell state and hidden state
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [ ]:
#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 50)           500050    
                                                                 
 dropout (Dropout)           (None, 300, 50)           0         
                                                                 
 lstm (LSTM)                 (None, 128)               91648     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 591,827
Trainable params: 91,777
Non-trainable params: 500,050
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train,
          epochs=5,
          batch_size=128,          
          validation_data=(X_test, y_test))

Epoch 1/5
94/94 [==============================] - 91s 949ms/step - loss: 0.6942 - accuracy: 0.5240 - val_loss: 0.6856 - val_accuracy: 0.5490
Epoch 2/5
94/94 [==============================] - 86s 913ms/step - loss: 0.6842 - accuracy: 0.5533 - val_loss: 0.6776 - val_accuracy: 0.5663
Epoch 3/5
94/94 [==============================] - 89s 945ms/step - loss: 0.6772 - accuracy: 0.5711 - val_loss: 0.6703 - val_accuracy: 0.5873
Epoch 4/5
94/94 [==============================] - 88s 936ms/step - loss: 0.6716 - accuracy: 0.5812 - val_loss: 0.6654 - val_accuracy: 0.5950
Epoch 5/5
94/94 [==============================] - 86s 914ms/step - loss: 0.6724 - accuracy: 0.5816 - val_loss: 0.6643 - val_accuracy: 0.5890


In [ ]:
model.fit(X_train,y_train,
          epochs=10,
          batch_size=128,          
          validation_data=(X_test, y_test))

Epoch 1/10
94/94 [==============================] - 88s 935ms/step - loss: 0.6685 - accuracy: 0.5913 - val_loss: 0.6719 - val_accuracy: 0.5870
Epoch 2/10
94/94 [==============================] - 89s 951ms/step - loss: 0.6645 - accuracy: 0.5936 - val_loss: 0.6602 - val_accuracy: 0.5983
Epoch 3/10
94/94 [==============================] - 86s 912ms/step - loss: 0.6577 - accuracy: 0.6067 - val_loss: 0.6608 - val_accuracy: 0.5993
Epoch 4/10
94/94 [==============================] - 87s 928ms/step - loss: 0.6548 - accuracy: 0.6113 - val_loss: 0.6661 - val_accuracy: 0.5897
Epoch 5/10
94/94 [==============================] - 85s 907ms/step - loss: 0.6516 - accuracy: 0.6110 - val_loss: 0.6580 - val_accuracy: 0.6107
Epoch 6/10
94/94 [==============================] - 88s 930ms/step - loss: 0.6471 - accuracy: 0.6214 - val_loss: 0.6568 - val_accuracy: 0.6073
Epoch 7/10
94/94 [==============================] - 87s 926ms/step - loss: 0.6427 - accuracy: 0.6242 - val_loss: 0.6566 - val_accuracy: 0.6053

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, classification_report, cohen_kappa_score
from sklearn import metrics 

In [ ]:
####### Prediction ##########
predictions = model.predict(X_test)

94/94 [==============================] - 14s 145ms/step


In [ ]:
predictions1 = [0 if x < 0.5 else 1 for x in predictions]

In [ ]:
print('Baseline: Accuracy: ', round(accuracy_score(y_test, predictions1)*100, 2))
print('\n Classification Report:\n', classification_report(y_test,predictions1))

Baseline: Accuracy:  61.3

 Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.67      0.63      1493
           1       0.63      0.55      0.59      1507

    accuracy                           0.61      3000
   macro avg       0.61      0.61      0.61      3000
weighted avg       0.61      0.61      0.61      3000

